In [151]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException


# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#beautifulsoup
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint

import re

import json

import pandas as pd

# 加入行為鍊 ActionChain (在 WebDriver 中模擬滑鼠移動、點擊、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains

# 加入鍵盤功能 (例如 Ctrl、Alt 等)
from selenium.webdriver.common.keys import Keys


In [152]:
# 處理日期的list 一組組搜尋寫成一個list這樣用index就可以搜尋了也比較好處理
all_split_date_combination_list = []
# 簡單生這樣就好[85,1,1, 6,30], [85,7,1,12,31]
for i in range(85, 114): # 民國85到113年
    month_day_list = [[1,1, 6,30], [7,1,12,31]]
    for b in month_day_list:
        b.insert(0, i)
        all_split_date_combination_list.append(b)

driver = webdriver.Chrome()
driver.get("https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx")

#等待頁面載入
try:
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
    (By.CSS_SELECTOR, 'input#jud_jmain')
    ))
    input_case_reason = driver.find_element(By.CSS_SELECTOR, 'input#jud_jmain')
    input_case_reason.send_keys("殺人")
    input_case_content = driver.find_element(By.CSS_SELECTOR, 'input#jud_kw')
    input_case_content.send_keys("中華民國刑法第 271 條&刑事判決")
    search_button = driver.find_element(By.CSS_SELECTOR, 'input#btnQry')
    search_button.click()
    sleep(1)
except TimeoutException:
    print("can not search")

In [153]:
driver = webdriver.Chrome()

In [154]:
# 放所有case的資料
all_case_list = []

# 現在的問題是，也些連一頁都沒有超過，那就要直接爬?
# 那就要重iframe抓網址，然後指指爬那頁
iframe = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-data")
all_case_list.append(iframe.get_attribute('src'))

all_case_list

one_page_url

In [155]:
# 每次查詢的總頁數網址
page_url_list = []

In [156]:
# 現在的問題是多頁時，無法正確的切換頁數

In [157]:
####這邊主程式

# 以切分日期的方式爬取所有資料
for date_split in all_split_date_combination_list:


    driver.get("https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx") # 每次搜完都要重新取得喔 #要寫在for裡面
    # 搜尋
    try:
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
        (By.CSS_SELECTOR, 'input#jud_jmain')
        ))
        input_case_reason = driver.find_element(By.CSS_SELECTOR, 'input#jud_jmain')
        input_case_reason.clear()   
        input_case_reason.send_keys("殺人")
        
        input_case_content = driver.find_element(By.CSS_SELECTOR, 'input#jud_kw')
        input_case_content.clear()
        input_case_content.send_keys("中華民國刑法第 271 條&刑事判決")
        
        input_date_from_year = driver.find_element(By.CSS_SELECTOR, 'input#dy1')
        input_date_from_month = driver.find_element(By.CSS_SELECTOR, 'input#dm1')
        input_date_from_day = driver.find_element(By.CSS_SELECTOR, 'input#dd1')

        input_date_end_year = driver.find_element(By.CSS_SELECTOR, 'input#dy2')
        input_date_end_month = driver.find_element(By.CSS_SELECTOR, 'input#dm2')
        input_date_end_day = driver.find_element(By.CSS_SELECTOR, 'input#dd2')

        input_date_from_year.clear()
        input_date_from_month.clear()
        input_date_from_day.clear()
        input_date_end_year.clear()
        input_date_end_month.clear()
        input_date_end_day.clear()

        # 開始輸入數字
        ac = ActionChains(driver)
        ac.send_keys_to_element(input_date_from_year, date_split[0])
        ac.send_keys_to_element(input_date_from_month, date_split[1])
        ac.send_keys_to_element(input_date_from_day, date_split[2])
        ac.send_keys_to_element(input_date_end_year, date_split[0])
        ac.send_keys_to_element(input_date_end_month, date_split[3])
        ac.send_keys_to_element(input_date_end_day, date_split[4])


        # 執行
        ac.perform()
        sleep(1)
        search_button = driver.find_element(By.CSS_SELECTOR, 'input#btnQry')
        search_button.click()
        sleep(1)
    except TimeoutException:
        print("can not search")
        continue
    
    #等待iframe與下一頁按鈕，計算pages
    only_one_page = False
    try:
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
        (By.CSS_SELECTOR, "iframe#iframe-data")
        ))

        iframe = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-data")
        one_page_url = iframe.get_attribute('src')  #切到iframe裡面就會抓不到了....要先抓....
        driver.switch_to.frame(iframe)
        # page_url_list = []
        # page_url_list.append(one_page_url) #切到iframe裡面就會抓不到了....要先抓....


        # 如果沒抓到這個，很可能只有一頁，那麼下面製造pages就不做了，程式也會在這裡斷開，交給下面有個if處理，若這種情況發生就去抓原本iframe的網址，只有一頁加入page_url_list這樣
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
        (By.CSS_SELECTOR, "a#hlNext")
        ))

        page_url = driver.find_element(By.CSS_SELECTOR, 'a#hlNext').get_attribute('href')
        # print(page_url)

        #計算頁數，與生成page_list
        how_many_page = driver.find_element(By.CSS_SELECTOR, "div#plPager span")
        page_re = "\d*(?=\s頁)"
        answer_page = re.search(page_re, how_many_page.text)
        page_url_list.clear()
        regex_page_url = "&page=[\d]*"
        test = re.split(regex_page_url, page_url)
        if int(answer_page[0]) <= 25:
            for each_page in range(1,int(answer_page[0])+1):
                page_url_list.append(f"{test[0]}&page={each_page}{test[1]}") 
        else:
            print("need to minimize split date interval, and data shortage")
            for each_page in range(1, 26):
                page_url_list.append(f"{test[0]}&page={each_page}{test[1]}")
            #這邊寫得好的話若超過25頁就要回到一開始的搜尋頁面，並把日期格式再切小
            # 妥協的辦法就是超過25就給他25頁就好

    except TimeoutException:
        only_one_page = True
        print("not found...next_page_button and pages")


    if only_one_page:
        page_url_list.clear()
        page_url_list.append(one_page_url) 



    for page in page_url_list:
        driver.get(page)

        # 要選日期舊到新，這樣時間才會順喔靠....選這個會重回第一頁.....哭阿.....還是就別選了到時候爬完用程式重排順序不就好了....
        # try:
        #     date_sequence_button_css = 'button#dropdown-menu-sort'
        #     WebDriverWait(driver, 10).until(
        #     EC.presence_of_element_located(
        #         (By.CSS_SELECTOR, date_sequence_button_css)
        #     ))

        #     date_sequence_button = driver.find_element(By.CSS_SELECTOR, date_sequence_button_css)
        #     date_sequence_button.click()
        # except TimeoutException:
        #     print("date sequence problem")

        # try:
            
        #     date_sequence_button_li_css = 'ul.dropdown-menu > li:nth-child(2)'
        #     WebDriverWait(driver, 10).until(
        #     EC.presence_of_element_located(
        #         (By.CSS_SELECTOR, date_sequence_button_li_css)
        #     ))

        #     li_button = driver.find_element(By.CSS_SELECTOR, date_sequence_button_li_css)
        #     li_button.click()
        # except TimeoutException:
        #     print("date sequence problem (options)")
        

        try:
            cssSelector = 'a#hlTitle'
            WebDriverWait(driver, 30).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, cssSelector)
            ))

            case_url = driver.find_elements(By.CSS_SELECTOR, cssSelector)
            
        except TimeoutException:
            print("cases not found") #我在想這可能根本抓不到，抓到就跳出怎會到click??
            continue

        # cases找到後開始一個一個點
        # 這是每一頁當中的至多20筆的資料    
        for case in case_url:
            try:
                case.click() #這裡有可能會報錯，原因滿多的，要看怎抓跟寫log #不能click可能也要refresh重抓之類的
            except Exception:
                print("this case can not click")
                continue

            try:
                # 看有沒有抓到js生成的側攔"當中的li"，li就是js後來生成的
                # 這邊要改可能還是沒有laws，而且結果超奇怪....根本抓不到law結果欄位有值，抓地到history但沒有值...
                
                side_li_selector_2 = 'div#JudHis div.panel-body ul li'
                WebDriverWait(driver, 15).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, side_li_selector_2)
                ))

                #還是這邊可以等兩個?等個兩個載好在來取得動態html
            except TimeoutException:
                # 如果有錯的話抓這個webelement的text(標題)，href(超連結)
                print(f"{case.text} {case.get_attribute('href')} not found...history, may be critical error")
                driver.refresh()
                side_li_selector_2 = 'div#JudHis div.panel-body ul li'
                WebDriverWait(driver, 15).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, side_li_selector_2)
                ))
                #這邊之後要改logging
            except StaleElementReferenceException:
                print("wrong!!!!!!!!!!!")
                driver.refresh()
                side_li_selector_2 = 'div#JudHis div.panel-body ul li'
                WebDriverWait(driver, 15).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, side_li_selector_2)
                ))

            try:   
                #不然這邊就是只看歷史審判(因為1.有時候很多所以加載比較慢2.好像一定會有此標籤因為至少也是"本案例無相關歷史案件"之類的)
                # 想一下這邊要用主動等待還是強制，主動等待的好處在於有可能是網路加載比較慢(可以避免這個問題)，而且真的沒這個欄位的時間應該很少
                # 但這個欄位真的有可能抓不到那就要抱錯，可是我們應該分開來報，因為兩個的問題不一樣
                # 這邊不只會有TimeoutException，還有其他的所以會抱錯嘗試refresh，重抓看看
                side_li_selector_1 = 'ul.rela-law li'
                WebDriverWait(driver, 10).until(    # 我還是不懂這邊怎麼會timeoutexception，不是用try except包起來了嗎????但之後可以嘗試看看F5抓新頁面的做法
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, side_li_selector_1)
                ))
            except TimeoutException:
                print(f"{case.text} {case.get_attribute('href')} not found...laws, it's ok")
                driver.refresh() # 如果等不到就refresh，畢竟這邊也只是再判斷有沒有而已
                side_li_selector_1 = 'ul.rela-law li'
                WebDriverWait(driver, 10).until(    # 我還是不懂這邊怎麼會timeoutexception，不是用try except包起來了嗎????但之後可以嘗試看看F5抓新頁面的做法
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, side_li_selector_1)
                ))
            except StaleElementReferenceException:
                print("wrong!!!!!!!!!!!")
                driver.refresh()
                side_li_selector_1 = 'ul.rela-law li'
                WebDriverWait(driver, 10).until(    # 我還是不懂這邊怎麼會timeoutexception，不是用try except包起來了嗎????但之後可以嘗試看看F5抓新頁面的做法
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, side_li_selector_1)
                ))


            #取得動態html #反正都要連到這個case當中，若上面兩個沒抓到，就只是side_bar的資料有少但其他資料還是有的
            #但我就在想這樣會不會很雞肋、而且反而拖到時間，因為有些case是真的沒有上面那兩個side_bar，還是乾脆就強制等待sleep算了，不知道哪種比較優且快
            sleep(1) # 稍微等一下laws那個side_bar位置
            case_html = driver.page_source
            soup = bs(case_html, "lxml")

            allSelector = 'div.col-td'
            all = soup.select(allSelector)

            # 這邊要加個判斷，不知道寫的可不可以，不是抓不到但好像怪怪的
            laws_list = []
            linesSelector = 'ul.rela-law li'
            lines = soup.select(linesSelector)
            for law in lines:
                laws_list.append(str(law.get_text()))

            
            history_judgement = soup.select("div#JudHis div.panel-body ul li")
            history_judgement_list = []
            for index in range(0, len(history_judgement)):
                if not str(history_judgement[index].select_one('a')) == "None":
                    history_judgement_list.append({
                        "case_id":str(history_judgement[index].get_text()),
                        "url":"https://judgment.judicial.gov.tw/FJUD/"+ str(history_judgement[index].select_one('a')['href'])
                    })
                else:
                    history_judgement_list.append({
                    "case_id":str(history_judgement[index].get_text())
                    })


            all_case_list.append({
            "case_id":str(all[0].get_text().strip()),
            "date":str(all[1].get_text().strip()),
            "simple_reason":str(all[2].get_text().strip()),
            "laws":laws_list,
            "history_judgement":history_judgement_list,
            "context":str(all[3].get_text()),
            })

            driver.back()
            # sleep(0.5) #迭完所有case後如果沒等一下就get新的page_url可能會造成下一頁沒有進去(這邊試試看，我也不知道)
            # 還是不行不知道為什麼好像有切頁但又跳回去了 每次切到新頁就會跳回第一頁，不知道為什麼，所以是有切過去的，但又跳回來???


    
    


not found...next_page_button and pages


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=124.0.6367.63); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF672A81502+60802]
	(No symbol) [0x00007FF6729FAC02]
	(No symbol) [0x00007FF6728B7CE4]
	(No symbol) [0x00007FF6728CA051]
	(No symbol) [0x00007FF6728CAA7A]
	(No symbol) [0x00007FF6728BE95B]
	(No symbol) [0x00007FF6728BC781]
	(No symbol) [0x00007FF6728C01DB]
	(No symbol) [0x00007FF6728C0280]
	(No symbol) [0x00007FF672900E0B]
	(No symbol) [0x00007FF67292AB7A]
	(No symbol) [0x00007FF6728FA7C6]
	(No symbol) [0x00007FF67292AD90]
	(No symbol) [0x00007FF67294A224]
	(No symbol) [0x00007FF67292A923]
	(No symbol) [0x00007FF6728F8FEC]
	(No symbol) [0x00007FF6728F9C21]
	GetHandleVerifier [0x00007FF672D8411D+3217821]
	GetHandleVerifier [0x00007FF672DC60B7+3488055]
	GetHandleVerifier 

In [ ]:
# 這個是對的，那位什麼會錯?上一頁沒設計好?
page_url_list

['https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=939c6c7c41d395e83a061bf7ba50b8cd&sort=DS&page=1&ot=in',
 'https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=939c6c7c41d395e83a061bf7ba50b8cd&sort=DS&page=2&ot=in',
 'https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=939c6c7c41d395e83a061bf7ba50b8cd&sort=DS&page=3&ot=in',
 'https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=939c6c7c41d395e83a061bf7ba50b8cd&sort=DS&page=4&ot=in',
 'https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=939c6c7c41d395e83a061bf7ba50b8cd&sort=DS&page=5&ot=in',
 'https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=939c6c7c41d395e83a061bf7ba50b8cd&sort=DS&page=6&ot=in',
 'https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=939c6c7c41d395e83a061bf7ba50b8cd&sort=DS&page=7&ot=in',
 'https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=939c6c7c41d395e83a061bf7ba50b8cd&sort=DS&page=8&ot=in',
 'https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=939c6c7c41d3

In [ ]:
len(all_case_list)

710

In [ ]:
print(all_case_list[44]['laws'])

['家庭暴力防治法 第 2、3 條（104.02.04）', '刑事訴訟法 第 156、158.2、158.4、159.1、159.2、159.5、299 條（106.11.16）', '中華民國刑法 第 1、25、38.1、95、271、320 條（105.11.30）']


In [ ]:
print(len(all_case_list))

1045


In [ ]:
# 就寫成dataframe再轉成csv八
tt = pd.DataFrame(all_case_list)
tt.to_csv('./all_cases_v1.csv')

In [ ]:
# side_bar兩個都要判斷?
# div#JudHis > div.panel-body > ul > li

In [ ]:
tt.info()

,case_id,date,simple_reason,laws,history_judgement,context
0,臺灣新北地方法院 113 年度訴字第 179 號刑事判決,民國 113 年 04 月 25 日,殺人未遂,"[刑事訴訟法 第 1、158.4、159.5、163.2、299 條（106.11.16）,...",[{'case_id': '臺灣新北地方法院 113 年度 訴 字第 179 號判決(113...,\n\n\n\n臺灣新北地方法院刑事判決113年度訴字第179號公 訴 人 臺灣新北地...
1,臺灣高等法院 112 年度上訴字第 4225 號刑事判決,民國 113 年 04 月 24 日,殺人未遂等,"[槍砲彈藥刀械管制條例 第 7、12 條（106.06.14）, 中華民國刑法 第 1、25...",[{'case_id': '臺灣新竹地方法院 112 年度 訴 字第 75 號裁定(112....,\n\n\n\n臺灣高等法院刑事判決112年度上訴字第4225號上 訴 人 臺灣新竹地...
2,臺灣高等法院 臺中分院 113 年度上訴字第 213 號刑事判決,民國 113 年 04 月 24 日,殺人未遂等,"[刑事訴訟法 第 159.1、369 條（106.11.16）, 中華民國刑法 第 1、13...",[{'case_id': '臺灣臺中地方法院 112 年度 訴 字第 1877 號裁定(11...,\n\n\n\n 臺灣高等法院臺中分院刑事判決113年度上訴字第213號上 訴 人 臺...
